# Isochrones for Melbourne and Christchurch

this note book uses the openrouteservice isochrones tool to get 400m walking distance for schools and supermarkets in christchurch and melbourne. To run this notebook, an Isochrones API key is required from https://openrouteservice.org/dev/#/home?tab=1


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
!pip install ipynb osmnx openrouteservice

In [9]:
import requests
import xml.etree.ElementTree as ET
import folium
import webbrowser
import time
import osmnx as ox
import geopandas as gpd
import openrouteservice.isochrones as ors
from os import path
from openrouteservice import client


# my API key:
api_key = '5b3ce3597851110001cf6248917c40c846744384a658cee39c9562d7'

# melb = 'City of Melbourne, Australia'
# melbourne = ox.geocode_to_gdf(melb)


In [10]:
# Melbourne city bounding box
bbox = [-37.850667, 144.896981,-37.775451, 144.991351]

url = f'https://overpass-api.de/api/interpreter?' \
      f'data=[out:json];' \
      f'(node["shop"="supermarket"]({",".join([str(x) for x in bbox])}););out;' \
      f'(way["shop"="supermarket"]({",".join([str(x) for x in bbox])}););out;' \
      f'(node["amenity"="school"]({",".join([str(x) for x in bbox])}););out;' \
      f'(way["amenity"="school"]({",".join([str(x) for x in bbox])}););out;' \
      f'>;' \
      f'out skel qt;'

r = requests.get(url)
melb_data = r.json()

In [12]:
# Create a dict to store all amenities of interest extracted from OSM
def get_data_subset(data):
  data_subset = {}

  for element in data['elements']:

      # Check that the element has a 'tags' dict and that the dict, if it exists, has the 'name' listed
      if 'tags' in element and 'name' in element['tags']:

          # If the element is a way, ie comprised of multiple nodes, then take the first node as the location
          if element['type'] == 'way':

              # for supermarkets, which are shops:
              if 'shop' in element['tags']:

                  data_subset[element['id']] = {
                  'amenity': element['tags']['shop']
                  , 'display name': element['tags']['name']
                  , 'type': 'way'
                  , 'location': element['nodes'][0]
              }

              # for hospitals and schools, which are amenities:
              elif 'amenity' in element['tags']:

                  data_subset[element['id']] = {
                  'amenity': element['tags']['amenity']
                  , 'display name': element['tags']['name']
                  , 'type': 'way'
                  , 'location': element['nodes'][0]
              }

            # If the element is a single node, store the location of the node
          elif element['type'] == 'node':

              # for supermarkets, which are shops:
              if 'shop' in element['tags']:

                  data_subset[element['id']] = {
                  'amenity': element['tags']['shop']
                  , 'display name': element['tags']['name']
                  , 'type': 'node'
                  , 'location': [element['lat'], element['lon']]
              }

              elif 'amenity' in element['tags']:

                  data_subset[element['id']] = {
                  'amenity': element['tags']['amenity']
                  , 'display name': element['tags']['name']
                  , 'type': 'node'
                  , 'location': [element['lat'], element['lon']]
              }

          else:
              print(f"Location {element['tags']['name']} does not appear to be readable")

  return data_subset


In [13]:
melb_data_subset = get_data_subset(melb_data)

In [15]:
# Extract the 'ways' from all amenities
def extract_way_locations(data_subset):
  way_nodes = [data_subset[key]['location'] for key in data_subset.keys() if data_subset[key]['type'] == 'way']

  way_locations = []

  # Query the API in batches of 100 nodes for speed.
  for i in range(0, len(way_nodes), 100):

      # Query the batch of nodes
      url = f"https://overpass-api.de/api/interpreter?" \
          f"node(id:{','.join(map(str, way_nodes[i:100 * (i + 1)]))});" \
          f"out;"
      r = requests.get(url)

      # XML tree output
      tree = ET.fromstring(r.text)
      way_locations.extend([x.attrib for x in tree.iter()][3:])
      return way_locations

In [16]:
melb_ways = extract_way_locations(melb_data_subset)

In [18]:
# function to get the ids of osm tags listed as ways
def get_way_ids(data_subset, way_locations):
  way_keys = [str(x['id']) for x in way_locations]  # Ensure all keys are strings

  # Assign the API response coordinates to the data
  for key in data_subset.keys():
      if data_subset[key]['type'] == 'way':
          location_str = str(data_subset[key]['location'])
          if location_str in way_keys:
              idx = way_keys.index(location_str)
              data_subset[key]['location'] = [way_locations[idx]['lat'], way_locations[idx]['lon']]
          else:
              # Handle the case where the location is not found in way_keys
              data_subset[key]['location'] = None  # or any other appropriate handling
  return data_subset

In [19]:
melb_data = get_way_ids(melb_data_subset, melb_ways)

{222420863: {'amenity': 'supermarket',
  'display name': 'Foodworks',
  'type': 'node',
  'location': [-37.7889872, 144.9760904]},
 302129542: {'amenity': 'supermarket',
  'display name': 'Woolworths',
  'type': 'node',
  'location': [-37.7981407, 144.9683576]},
 370224122: {'amenity': 'supermarket',
  'display name': 'North Melbourne IGA X-press',
  'type': 'node',
  'location': [-37.798856, 144.953525]},
 506153647: {'amenity': 'supermarket',
  'display name': 'Woolworths',
  'type': 'node',
  'location': [-37.8205606, 144.9436054]},
 573818021: {'amenity': 'supermarket',
  'display name': 'McCoppins Food & Wine',
  'type': 'node',
  'location': [-37.7888604, 144.9895506]},
 580722996: {'amenity': 'supermarket',
  'display name': 'KimChi Korean Japanese Grocery',
  'type': 'node',
  'location': [-37.802712, 144.977554]},
 591382457: {'amenity': 'supermarket',
  'display name': 'Coles',
  'type': 'node',
  'location': [-37.8175611, 144.9649673]},
 595501090: {'amenity': 'supermarket',

In [20]:
# Generate the API parameters. using networks that are acessible by foot according to OSM
# Range of distance to match ideal walkind distances according to literature
params = {
    'profile': 'foot-walking'
    , 'range_type': 'distance'
    , 'range': [800]  #  800 m isochrones
}

# Throttling requests to ensure API is not overloaded
throttle_requests = True

In [21]:

def make_isochrones(data_subset, api_key, params):
# The API key is entered at the top of this notebook
  ors = client.Client(key=api_key)

  # Loop over the amentites and query the API for the isochrone data, then add each amenity to the map
  for key in data_subset.keys():

      # sleep every three seconds to ensure query parameters are not exceeded (limit 20 per minute)
      if throttle_requests:
          time.sleep(3)

      # Add amenity coordinates to params and query ORS.
      params['locations'] = [list(reversed(data_subset[key]['location']))]
      data_subset[key]['isochrone'] = ors.isochrones(**params)
  return data_subset


In [22]:

#@title The following block of code is computationally expensive, and requires an API key to run. To save time, the melbourne amenity isochrones have been saved as .pkl file in the data section of this folder.

In [23]:
# UNCOMMENT TO RUN
# melb_isochrones = make_isochrones(melb_data, api_key, params)

In [24]:
# save isochrone data subset as an object
import pickle

with open('/content/drive/MyDrive/FinalProject/Data/amenity-isochrones/melbourne_amenity_isochrones.pkl', 'wb') as f:
    pickle.dump(melb_isochrones, f)


In [26]:
# Layer Groups
def map_amenities(data_subset, m):
  sch_layer = folium.FeatureGroup(name='Schools')
  sup_layer = folium.FeatureGroup(name='Supermarkets')
  for key in data_subset.keys():

      if 'isochrone' in data_subset[key] and data_subset[key]['amenity'] == 'supermarket':
      # Add isochrone to the map
          folium.features.GeoJson(
              data=data_subset[key]['isochrone']
              , style_function=lambda feature: dict(color='#2C7329')
          ).add_to(sup_layer)

      elif 'isochrone' in data_subset[key] and data_subset[key]['amenity'] == 'school':
      # Add isochrone to the map
          folium.features.GeoJson(
              data=data_subset[key]['isochrone']
              , style_function=lambda feature: dict(color='#3171AE')
          ).add_to(sch_layer)

      else:
          continue
  sch_layer.add_to(m)
  sup_layer.add_to(m)
  return m #sch_layer, sup_layer





# m.save('Amenities_melbourne.html')
# webbrowser.open(path.realpath('Amenities_melbourne.html'), new=2)

In [ ]:
import matplotlib.pyplot as plt
import folium
import branca.colormap as cm

# function to produce a map of greenspace and their values

def greenspace_map(gdf, latitude, longitude, col_name):
    try:
        m = folium.Map(location=[latitude, longitude], zoom_start=12, tiles='cartodb positron')
        gdf.explore(gdf[col_name],
                    cmap=['#5c6b28','#7c9d45','#a47e4f'],
                    tooltip=col_name,
                    m=m,
                    popup=f"{gdf['park_name']}"
                f" Average NDVI value: {gdf['median_NDVI']}",
                    categorical=True)

        return m
    except Exception as e:
        return str(e)

In [2]:
chc = 'Christchurch, New Zealand'
christchurch = ox.geocode_to_gdf(chc)
christchurch.bounds

NameError: ignored

In [30]:
# Christchurch city bounding box
bbox = [-43.629201, 172.393025, -43.389087, 172.821627]

url = f'https://overpass-api.de/api/interpreter?' \
      f'data=[out:json];' \
      f'(node["shop"="supermarket"]({",".join([str(x) for x in bbox])}););out;' \
      f'(way["shop"="supermarket"]({",".join([str(x) for x in bbox])}););out;' \
      f'(node["amenity"="school"]({",".join([str(x) for x in bbox])}););out;' \
      f'(way["amenity"="school"]({",".join([str(x) for x in bbox])}););out;' \
      f'>;' \
      f'out skel qt;'

r = requests.get(url)
chc_data = r.json()

In [31]:
chc_data_subset = get_data_subset(chc_data)
chc_ways = extract_way_locations(chc_data_subset)
chc_data_subset = get_way_ids(chc_data_subset, chc_ways)

In [1]:
# Removing keys from the data where no location is tagged
keys_to_remove = []

# Loop through the dictionary and identify keys to remove
for key, value in chc_data_subset.items():
    if 'location' in value and value['location'] == 'none':
        keys_to_remove.append(key)

# Remove the identified keys
for key in keys_to_remove:
    del chc_data_subset[key]

NameError: ignored

In [32]:
chch_isochrones = make_isochrones(chc_data_subset, api_key, params)

TypeError: ignored

In [35]:
# save isochrone data subset as an object
import pickle

with open('/content/drive/MyDrive/FinalProject/Data/amenity-isochrones/christchurch_amenity_isochrones.pkl', 'wb') as f:
    pickle.dump(chc_data_subset, f)